# Code processes and merges the two datasets scraped from Food.com 

In [112]:
import pandas as pd
import re
import numpy as np

In [75]:
df1 = pd.read_csv('Datasets/recipes_ingredients.csv')
df2 = pd.read_csv('Datasets/recipes.csv')

In [76]:
df1.head()

,id,name,description,ingredients,ingredients_raw,steps,servings,serving_size,tags
0,71247,Cherry Streusel Cobbler,"I haven't made this in years, so I'm just gues...","[""cherry pie filling"", ""condensed milk"", ""melt...","[""2 (21 ounce) cans cherry pie filling"",""2...","[""Preheat oven to 375°F."", ""Spread cherry pie ...",6.0,1 (347 g),"[""60-minutes-or-less"", ""time-to-make"", ""course..."
1,76133,Reuben and Swiss Casserole Bake,I think this is even better than a reuben sand...,"[""corned beef chopped"", ""sauerkraut cold water...","[""1/2-1 lb corned beef, cooked and choppe...","[""Set oven to 350 degrees F."", ""Butter a 9 x 1...",4.0,1 (207 g),"[""60-minutes-or-less"", ""time-to-make"", ""course..."
2,503816,Yam-Pecan Recipe,A lady I work with heard me taking about ZWT a...,"[""unsalted butter"", ""vegetable oil"", ""all - pu...","[""3/4 cup unsalted butter, at room tempera...","[""Preheat oven to 350°F In a mixing bowl, usi...",8.0,1 (198 g),"[""time-to-make"", ""course"", ""main-ingredient"", ..."
3,418749,Tropical Orange Layer Cake,An easy and delicious cake. Great for a summ...,"[""orange cake mix"", ""instant vanilla pudding"",...","[""1 (18 ounce) pkge.orange cake mix"",""1 (3...","[""In a large mixing bowl, combine the first 6 ...",16.0,1 (191 g),"[""60-minutes-or-less"", ""time-to-make"", ""course..."
4,392934,Safe to Eat Raw Chocolate Chip Oreo Cookie &qu...,I was searching the web for something like thi...,"[""butter"", ""brown sugar"", ""granulated sugar"", ...","[""1/2 cup butter, room temperature "",""1/2 ...","[""Cream butter and sugars together."", ""Blend i...",24.0,1 (26 g),"[""15-minutes-or-less"", ""time-to-make"", ""course..."


In [77]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500471 entries, 0 to 500470
Data columns (total 9 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   id               500471 non-null  int64  
 1   name             500471 non-null  object 
 2   description      498285 non-null  object 
 3   ingredients      500471 non-null  object 
 4   ingredients_raw  500436 non-null  object 
 5   steps            500471 non-null  object 
 6   servings         499749 non-null  float64
 7   serving_size     500436 non-null  object 
 8   tags             500436 non-null  object 
dtypes: float64(1), int64(1), object(7)
memory usage: 34.4+ MB


In [78]:
df2.head()

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions
0,38,Low-Fat Berry Blue Frozen Dessert,1533,Dancer,PT24H,PT45M,PT24H45M,1999-08-09T21:46:00Z,Make and share this Low-Fat Berry Blue Frozen ...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,1.3,8.0,29.8,37.1,3.6,30.2,3.2,4.0,NaN,"c(""Toss 2 cups berries with sugar."", ""Let stan..."
1,39,Biryani,1567,elly9812,PT25M,PT4H,PT4H25M,1999-08-29T13:12:00Z,Make and share this Biryani recipe from Food.com.,"c(""https://img.sndimg.com/food/image/upload/w_...",...,16.6,372.8,368.4,84.4,9.0,20.4,63.4,6.0,NaN,"c(""Soak saffron in warm milk for 5 minutes and..."
2,40,Best Lemonade,1566,Stephen Little,PT5M,PT30M,PT35M,1999-09-05T19:52:00Z,This is from one of my first Good House Keepi...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,0.0,0.0,1.8,81.5,0.4,77.2,0.3,4.0,NaN,"c(""Into a 1 quart Jar with tight fitting lid, ..."
3,41,Carina's Tofu-Vegetable Kebabs,1586,Cyclopz,PT20M,PT24H,PT24H20M,1999-09-03T14:54:00Z,This dish is best prepared a day in advance to...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,3.8,0.0,1558.6,64.2,17.3,32.1,29.3,2.0,4 kebabs,"c(""Drain the tofu, carefully squeezing out exc..."
4,42,Cabbage Soup,1538,Duckie067,PT30M,PT20M,PT50M,1999-09-19T06:19:00Z,Make and share this Cabbage Soup recipe from F...,"""https://img.sndimg.com/food/image/upload/w_55...",...,0.1,0.0,959.3,25.1,4.8,17.7,4.3,4.0,NaN,"c(""Mix everything together and bring to a boil..."


In [79]:
df2.columns

Index(['RecipeId', 'Name', 'AuthorId', 'AuthorName', 'CookTime', 'PrepTime',
       'TotalTime', 'DatePublished', 'Description', 'Images', 'RecipeCategory',
       'Keywords', 'RecipeIngredientQuantities', 'RecipeIngredientParts',
       'AggregatedRating', 'ReviewCount', 'Calories', 'FatContent',
       'SaturatedFatContent', 'CholesterolContent', 'SodiumContent',
       'CarbohydrateContent', 'FiberContent', 'SugarContent', 'ProteinContent',
       'RecipeServings', 'RecipeYield', 'RecipeInstructions'],
      dtype='object')

In [80]:
# Remove double spaces in columns
df1['name'] = df1['name'].str.replace(r'\s{2,}', ' ', regex = True)
df2['name'] = df1['name'].str.replace(r'\s{2,}', ' ', regex = True)
df1['ingredients_raw'] = df1['ingredients_raw'].str.replace(r'\s{2,}', ' ', regex = True)

# Remove qutotations in names
df1['name'] = df1['name'].str.replace('"', '', regex = False)
df2['name'] = df2['name'].str.replace('"', '', regex = False)

In [81]:
# Cut down data frames
df1 = df1[['id','name', 'description', 'ingredients','ingredients_raw', 'steps', 'servings']]
df2 = df2[['RecipeId','Name', 'TotalTime', 'Calories', 'FatContent', 'SaturatedFatContent', 'CholesterolContent', 'SodiumContent', 'CarbohydrateContent', 'FiberContent', 'SugarContent', 'ProteinContent']]
renamed_df1 = df1.rename(columns = {'id' : 'ID',
                            'name': 'Name',
                            'description' : 'Description',
                            'ingredients' : 'IngredientsExtracted',
                            'ingredients_raw' : 'IngredientsRaw',
                            'steps' : 'Instructions',
                            'servings' : 'Servings'})

In [82]:
# Inner merge, dropping duplicates and null values in Ingredients
merged_df = pd.merge(renamed_df1, df2, how = 'inner', left_on = 'ID', right_on = 'RecipeId')
# drop one of the name and id columns and rename remaining one
merged_df = merged_df.drop(columns=['Name_y','RecipeId'])
merged_df = merged_df.rename(columns = {'Name_x' : 'Name'})
# drop duplicate rows and rows w/ null ingredients
merged_df = merged_df.drop_duplicates(subset = 'ID', keep = 'first')
merged_df = merged_df.dropna(subset = ['IngredientsExtracted','IngredientsRaw']).reset_index(drop=True)

# Change to amp and quotation mark
merged_df['Name'] = merged_df['Name'].replace({'&amp;': '&', '&quot;': '"', '&quot': '"', '&ndash': ''}, regex = True)
merged_df['Instructions'] = merged_df['Instructions'].replace({'&amp;': '&', '&quot;': '"', '&quot': '"', '&ndash': ''}, regex = True)

In [83]:
# remove punctuation from IngredientsExtracted column
def clean_ingredients(ingredients_list):
    # join the list into a single string
    ingredients_str = ''.join(ingredients_list)
    # remove brackets, quotes, hyphens, and additional punctuation
    cleaned_ingredients = re.sub(r'[\"\[\]\-\,]', '', ingredients_str)
    return cleaned_ingredients

# apply the function
merged_df['IngredientsExtracted'] = merged_df['IngredientsExtracted'].apply(clean_ingredients)

In [84]:
merged_df.head()

,ID,Name,Description,IngredientsExtracted,IngredientsRaw,Instructions,Servings,TotalTime,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent
0,71247,Cherry Streusel Cobbler,"I haven't made this in years, so I'm just gues...",cherry pie filling condensed milk melted marga...,"[""2 (21 ounce) cans cherry pie filling"",""2 egg...","[""Preheat oven to 375°F."", ""Spread cherry pie ...",6.0,PT1H,801.0,29.1,7.7,93.0,536.5,125.0,3.3,54.4,12.3
1,76133,Reuben and Swiss Casserole Bake,I think this is even better than a reuben sand...,corned beef chopped sauerkraut cold water swis...,"[""1/2-1 lb corned beef, cooked and chopped "",""...","[""Set oven to 350 degrees F."", ""Butter a 9 x 1...",4.0,PT40M,664.4,45.3,22.1,142.3,2074.2,33.9,5.9,7.1,31.0
2,503816,Yam-Pecan Recipe,A lady I work with heard me taking about ZWT a...,unsalted butter vegetable oil all purpose flo...,"[""3/4 cup unsalted butter, at room temperature...","[""Preheat oven to 350°F In a mixing bowl, usi...",8.0,PT1H15M,956.8,53.2,21.7,192.5,664.9,112.8,4.1,63.3,10.7
3,418749,Tropical Orange Layer Cake,An easy and delicious cake. Great for a summ...,orange cake mix instant vanilla pudding orange...,"[""1 (18 ounce) pkge.orange cake mix"",""1 (3 oun...","[""In a large mixing bowl, combine the first 6 ...",16.0,PT40M,581.6,30.5,17.5,62.4,361.5,74.6,3.5,61.8,6.2
4,392934,"Safe to Eat Raw Chocolate Chip Oreo Cookie ""do...",I was searching the web for something like thi...,butter brown sugar granulated sugar milk vanil...,"[""1/2 cup butter, room temperature "",""1/2 cup ...","[""Cream butter and sugars together."", ""Blend i...",24.0,PT15M,121.4,5.9,3.3,10.5,77.8,16.8,0.5,10.3,1.1


In [114]:
# replace blank or whitespace only strings with NaN
merged_df['IngredientsExtracted'] = merged_df['IngredientsExtracted'].replace(r'^\s*$', np.nan, regex=True)

# drop rows where IngredientsExtracted has NaN
merged_df = merged_df.dropna(subset='IngredientsExtracted')

In [85]:
# Function to convert the TotalTime string to minutes
def convert_to_minutes(time):
    hours = 0
    minutes = 0
    
    hours_match = re.search(r'(\d+)H', time)

    if hours_match:
        hours = int(hours_match.group(1))
    
    minutes_match = re.search(r'(\d+)M', time)

    if minutes_match:
        minutes = int(minutes_match.group(1))
    
    total_minutes = hours * 60 + minutes
    return total_minutes

# Apply function
merged_df['TotalTime'] = merged_df['TotalTime'].apply(convert_to_minutes)

In [86]:
merged_df.head()

,ID,Name,Description,IngredientsExtracted,IngredientsRaw,Instructions,Servings,TotalTime,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent
0,71247,Cherry Streusel Cobbler,"I haven't made this in years, so I'm just gues...",cherry pie filling condensed milk melted marga...,"[""2 (21 ounce) cans cherry pie filling"",""2 egg...","[""Preheat oven to 375°F."", ""Spread cherry pie ...",6.0,60,801.0,29.1,7.7,93.0,536.5,125.0,3.3,54.4,12.3
1,76133,Reuben and Swiss Casserole Bake,I think this is even better than a reuben sand...,corned beef chopped sauerkraut cold water swis...,"[""1/2-1 lb corned beef, cooked and chopped "",""...","[""Set oven to 350 degrees F."", ""Butter a 9 x 1...",4.0,40,664.4,45.3,22.1,142.3,2074.2,33.9,5.9,7.1,31.0
2,503816,Yam-Pecan Recipe,A lady I work with heard me taking about ZWT a...,unsalted butter vegetable oil all purpose flo...,"[""3/4 cup unsalted butter, at room temperature...","[""Preheat oven to 350°F In a mixing bowl, usi...",8.0,75,956.8,53.2,21.7,192.5,664.9,112.8,4.1,63.3,10.7
3,418749,Tropical Orange Layer Cake,An easy and delicious cake. Great for a summ...,orange cake mix instant vanilla pudding orange...,"[""1 (18 ounce) pkge.orange cake mix"",""1 (3 oun...","[""In a large mixing bowl, combine the first 6 ...",16.0,40,581.6,30.5,17.5,62.4,361.5,74.6,3.5,61.8,6.2
4,392934,"Safe to Eat Raw Chocolate Chip Oreo Cookie ""do...",I was searching the web for something like thi...,butter brown sugar granulated sugar milk vanil...,"[""1/2 cup butter, room temperature "",""1/2 cup ...","[""Cream butter and sugars together."", ""Blend i...",24.0,15,121.4,5.9,3.3,10.5,77.8,16.8,0.5,10.3,1.1


In [87]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 490458 entries, 0 to 490457
Data columns (total 17 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   ID                    490458 non-null  int64  
 1   Name                  490458 non-null  object 
 2   Description           488272 non-null  object 
 3   IngredientsExtracted  490458 non-null  object 
 4   IngredientsRaw        490458 non-null  object 
 5   Instructions          490458 non-null  object 
 6   Servings              490019 non-null  float64
 7   TotalTime             490458 non-null  int64  
 8   Calories              490458 non-null  float64
 9   FatContent            490458 non-null  float64
 10  SaturatedFatContent   490458 non-null  float64
 11  CholesterolContent    490458 non-null  float64
 12  SodiumContent         490458 non-null  float64
 13  CarbohydrateContent   490458 non-null  float64
 14  FiberContent          490458 non-null  float64
 15  

In [116]:
merged_df.isna().sum()

ID                      0
Name                    0
Description             0
IngredientsExtracted    0
IngredientsRaw          0
Instructions            0
Servings                4
TotalTime               0
Calories                0
FatContent              0
SaturatedFatContent     0
CholesterolContent      0
SodiumContent           0
CarbohydrateContent     0
FiberContent            0
SugarContent            0
ProteinContent          0
dtype: int64

In [115]:
# export csv
merged_df.to_csv('recipes_food_com.csv', index = False)